<a href="https://colab.research.google.com/github/AfrozSaqlain/Physics-Informed-Neural-Network/blob/main/QNM_of_Kerr_BH.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim

import numpy as np
import matplotlib.pyplot as plt

### Future Modifications

Need to implement Chebyshev polynomials for sampling

In [2]:
x = torch.linspace(0, 1, 100).view(-1, 1).requires_grad_()
u = torch.linspace(-1, 1, 100).view(-1, 1).requires_grad_()

\begin{align}
F_0(a, x, m, s, \omega, r_+, A) &= -a^4 x^2 \omega^2 - 2 a^3 m x^2 \omega + a^2 \bigg(-A x^2 + x^2 \Big(4 (r_+ + 1)^\omega{}^2 + 2j (r_+ + 2) \omega + 2j s (\omega + 1j) - 2\Big) \nonumber \\
&\quad + x \omega^2 - \omega^2\bigg) + 2 a m \Big(r_+ x^2 (2 \omega + 1j) - x (\omega + 1j) - \omega\Big) \nonumber \\
&\quad + A (x - 1) - 1j r_+ (2 \omega + 1j) \Big(x^2 (s - 2 1j \omega + 1) - 2 (s + 1) x + 2 1j \omega\Big) \nonumber \\
&\quad + (s + 1) (x - 2j \omega).
\end{align}

\begin{align}
F_1(a, x, m, s, \omega, r_+) &= 2 a^4 x^4 (x - 1j \omega) - 2j a^3 m x^4 + a^2 x^2 \Big(2 r_+ x^2 (-1 + 2j \omega) - (s + 3) x^2 + 2 x (s + 1j \omega + 2) - 4j \omega\Big) \nonumber \\
&\quad + 2j a m (x - 1) x^2 + (x - 1) \Big(2 r_+ x^2 (1 - 2j \omega) + (s + 1) x^2 - 2 (s + 1) x + 2j \omega\Big).
\end{align}

\begin{align}
F_2(a, x) &= a^4 x^6 - 2 a^2 (x - 1) x^4 + (x - 1)^2 x^2.
\end{align}

\begin{align}
G_0(a, u, m, s, \omega, A) &= 4 a^2 (u^2 - 1) \omega^2 - 4 a (u^2 - 1) \omega \Big((u - 1) |m - s| + (u + 1) |m + s| + 2 (s + 1) u\Big) \nonumber \\
&\quad + 4 \Big(A (u^2 - 1) + m^2 + 2 m s u + s \big((s + 1) u^2 - 1\big)\Big) \nonumber \\
&\quad - 2 (u^2 - 1) |m + s| - 2 (u^2 - 1) |m - s| (|m + s| + 1) \nonumber \\
&\quad - (u - 1)^2 |m - s|^2 - (u + 1)^2 |m + s|^2.
\end{align}

\begin{align}
G_1(a, u, m, s, \omega) &= -8 a (u^2 - 1)^2 \omega - 4 (u^2 - 1) \Big((u - 1) |m - s| + (u + 1) |m + s| + 2 u\Big).
\end{align}

\begin{align}
G_2(u) &= -4 (u^2 - 1)^2.
\end{align}


In [3]:
def F_0(a, x, m, s, omega, r_plus, A):
    return -a**4 * x**2 * omega**2 - 2 * a**3 *m * x**2 * omega + a**2 * (-A * x**2 + x**2 * (4 * (r_plus + 1) ** omega**2 + 2j * (r_plus + 2) * omega + 2j * s * (omega + 1j) - 2) + x * omega**2 - omega**2) + 2 * a * m * (r_plus * x**2 * (2 * omega + 1j) - x * (omega + 1j) - omega) + A * (x - 1) - 1j * r_plus * (2 * omega + 1j) * (x**2 * (s - 2 * 1j * omega + 1) - 2 * (s + 1) * x + 2 * 1j * omega) + (s + 1) * (x - 2j * omega)

def F_1(a, x, m, s, omega, r_plus):
    return 2 * a**4 * x**4 * (x - 1j * omega) - 2j * a**3 * m * x**4 + a**2 * x**2 * (2 * r_plus * x**2 * (-1 +2j * omega) - (s + 3) * x**2 + 2 * x * (s + 1j * omega + 2) - 4j * omega) + 2j * a * m * (x - 1) * x**2 + (x - 1) * (2 * r_plus * x**2 * (1 - 2j * omega) + (s + 1) * x**2 - 2 * (s + 1) * x + 2j * omega)

def F_2(a, x):
    return a**4 * x**6 - 2 * a**2 * (x - 1) * x**4 + (x - 1)**2 * x**2

def G_0(a, u, m, s, omega, A):
    return 4 * a**2 * (u**2 - 1) * omega**2 - 4 * a * (u**2 - 1) * omega * ((u - 1) * torch.abs(torch.tensor(m - s)) + (u + 1) * torch.abs(torch.tensor(m + s)) + 2 * (s + 1) * u) + 4 * (A * (u**2 - 1) + m**2 + 2 * m * s * u + s * ((s + 1) * u**2 - 1)) - 2 * (u**2 - 1) * torch.abs(torch.tensor(m + s)) - 2 * (u**2 - 1) * torch.abs(torch.tensor(m - s)) * (torch.abs(torch.tensor(m + s)) + 1) - (u - 1)**2 * (torch.abs(torch.tensor(m - s)))**2 - (u + 1)**2 * (torch.abs(torch.tensor(m + s)))**2

def G_1(a, u, m, s, omega):
    return -8 * a * (u**2 - 1)**2 * omega - 4 * (u**2 - 1) * ((u - 1) * torch.abs(torch.tensor(m - s)) + (u + 1) * torch.abs(torch.tensor(m + s)) + 2 * u)

def G_2(u):
    return -4 * (u**2 - 1)**2

In [4]:
class QNM_radial(nn.Module):
    def __init__(self, input_size = 1, hidden_size = 200, output_size = 2):
        super(QNM_radial, self).__init__()
        self.layer = nn.Sequential(
            nn.Linear(input_size, hidden_size),
            nn.Tanh(),
            nn.Linear(hidden_size, hidden_size),
            nn.Tanh(),
            nn.Linear(hidden_size, output_size)
        )

    def forward(self, x):
        output = (torch.exp(x - 1) - 1) * self.layer(x) + 1
        return output

class QNM_angular(nn.Module):
    def __init__(self, input_size = 1, hidden_size = 200, output_size = 2):
        super(QNM_angular, self).__init__()
        self.layer = nn.Sequential(
            nn.Linear(input_size, hidden_size),
            nn.Tanh(),
            nn.Linear(hidden_size, hidden_size),
            nn.Tanh(),
            nn.Linear(hidden_size, output_size)
        )

    def forward(self, u):
        output = (torch.exp(u + 1) - 1) * self.layer(u) + 1
        return output

omega = nn.Parameter(torch.tensor([0.7, -0.1], requires_grad=True))
# A = nn.Parameter(torch.tensor(1.0, requires_grad=True))

## Parameters for Schwarzschild BH case (with gravitational perturbation)

- a = 0 (spinless)
- s = -2
- m = 0
- $r_+ = \frac{1 + \sqrt{1 - 4 a^2}}{2}$

In [5]:
a, s, m= 0, -2, 0
r_plus = (1 + np.sqrt(1 - 4 * a**2)) / 2
l = 2
A = l * (l + 1) - s * (s + 1)

In [6]:
net1 = QNM_radial()
net2 = QNM_angular()

opt_net1 = torch.compile(net1)
opt_net2 = torch.compile(net2)

# optimizer = optim.Adam(list(net1.parameters()) + list(net2.parameters()) + [omega, A], lr=0.005)
optimizer = optim.Adam(list(net1.parameters()) + list(net2.parameters()) + [omega], lr=0.005)
scheduler = optim.lr_scheduler.ExponentialLR(optimizer, 0.999)

In [7]:
def loss(weight):
    f_r, f_i = net1(x)[:, 0].unsqueeze(dim = 1), net1(x)[:, 1].unsqueeze(dim = 1)
    g_r, g_i = net2(u)[:, 0].unsqueeze(dim = 1), net2(u)[:, 1].unsqueeze(dim = 1)

    f_r_x = torch.autograd.grad(f_r, x, grad_outputs=torch.ones_like(f_r), create_graph=True)[0]
    f_r_xx = torch.autograd.grad(f_r_x, x, grad_outputs=torch.ones_like(f_r_x), create_graph=True)[0]

    f_i_x = torch.autograd.grad(f_i, x, grad_outputs=torch.ones_like(f_i), create_graph=True)[0]
    f_i_xx = torch.autograd.grad(f_i_x, x, grad_outputs=torch.ones_like(f_i_x), create_graph=True)[0]

    g_r_u = torch.autograd.grad(g_r, u, grad_outputs=torch.ones_like(g_r), create_graph=True)[0]
    g_r_uu = torch.autograd.grad(g_r_u, u, grad_outputs=torch.ones_like(g_r_u), create_graph=True)[0]

    g_i_u = torch.autograd.grad(g_i, u, grad_outputs=torch.ones_like(g_i), create_graph=True)[0]
    g_i_uu = torch.autograd.grad(g_i_u, u, grad_outputs=torch.ones_like(g_i_u), create_graph=True)[0]

    F_0_ = F_0(a = a, x = x, m = m, s = s, omega = torch.complex(omega[0], omega[1]), r_plus = r_plus, A = A)
    F_1_ = F_1(a = a, x = x, m = m, s = s, omega = torch.complex(omega[0], omega[1]), r_plus = r_plus)
    F_2_ = F_2(a = a, x = x)

    G_0_ = G_0(a = a, u = u, m = m, s = s, omega = torch.complex(omega[0], omega[1]), A = A)
    G_1_ = G_1(a = a, u = u, m = m, s = s, omega = torch.complex(omega[0], omega[1]))
    G_2_ = G_2(u = u)

    L_F_  = torch.abs(torch.complex(F_2_ * f_r_xx + F_1_.real * f_r_x - F_1_.imag * f_i_x + F_0_.real * f_r - F_0_.imag * f_i, F_2_ * f_i_xx + F_1_.real * f_i_x + F_1_.imag * f_r_x + F_0_.real * f_i + F_0_.imag * f_r))
    L_G_ = torch.abs(torch.complex(G_2_ * g_r_uu + G_1_.real * g_r_u - G_1_.imag * g_i_u + G_0_.real * g_r - G_0_.imag * g_i, G_2_ * g_i_uu + G_1_.real * g_i_u + G_1_.imag * g_r_u + G_0_.real * g_i + G_0_.imag * g_r))

    return weight * torch.mean(L_F_) + torch.mean(L_G_)


In [8]:
for i in range(2000):
    optimizer.zero_grad()
    loss_ = loss(10)
    loss_.backward()
    # print(omega.grad, A.grad)
    optimizer.step()
    scheduler.step()

    if i % 100 == 0:
        print(f"Epoch {i}, Loss: {loss_}")

Epoch 0, Loss: 24.5294246673584
Epoch 100, Loss: 0.9592694044113159
Epoch 200, Loss: 0.8667488694190979
Epoch 300, Loss: 0.6623705625534058
Epoch 400, Loss: 0.4481692910194397
Epoch 500, Loss: 0.7105443477630615
Epoch 600, Loss: 0.5770654678344727
Epoch 700, Loss: 0.4662024974822998
Epoch 800, Loss: 0.4149976968765259
Epoch 900, Loss: 0.881445586681366
Epoch 1000, Loss: 0.3525920510292053
Epoch 1100, Loss: 0.235138401389122
Epoch 1200, Loss: 0.22144412994384766
Epoch 1300, Loss: 0.1829904317855835
Epoch 1400, Loss: 0.14509275555610657
Epoch 1500, Loss: 0.28139728307724
Epoch 1600, Loss: 0.1440720558166504
Epoch 1700, Loss: 0.20608071982860565
Epoch 1800, Loss: 0.19573983550071716
Epoch 1900, Loss: 0.14794808626174927


In [9]:
omega

Parameter containing:
tensor([ 0.7474, -0.1842], requires_grad=True)

The true value for $\omega = 0.74734 - i 17793$ according to Leaver's method.

### For (l, m, n) = (3, 3, 0)

In [10]:
a, s, m= 0, -2, 3
r_plus = (1 + np.sqrt(1 - 4 * a**2)) / 2
l = 3
A = l * (l + 1) - s * (s + 1)

In [11]:
optimizer = optim.Adam(list(net1.parameters()) + list(net2.parameters()) + [omega], lr=0.005)
scheduler = optim.lr_scheduler.ExponentialLR(optimizer, 0.999)


for i in range(2500):
    optimizer.zero_grad()
    loss_ = loss(10)
    loss_.backward()
    # print(omega.grad, A.grad)
    optimizer.step()
    scheduler.step()

    if i % 200 == 0:
        print(f"Epoch {i}, Loss: {loss_}")

Epoch 0, Loss: 29.963539123535156
Epoch 200, Loss: 2.706085205078125
Epoch 400, Loss: 0.9042857885360718
Epoch 600, Loss: 0.674662172794342
Epoch 800, Loss: 1.2629833221435547
Epoch 1000, Loss: 0.6053053140640259
Epoch 1200, Loss: 0.41230887174606323
Epoch 1400, Loss: 0.3847203552722931
Epoch 1600, Loss: 0.2793304920196533
Epoch 1800, Loss: 0.2515694797039032
Epoch 2000, Loss: 0.18043601512908936
Epoch 2200, Loss: 0.17966563999652863
Epoch 2400, Loss: 0.139055997133255


In [12]:
omega

Parameter containing:
tensor([ 1.2624, -0.5707], requires_grad=True)

The value for QNM frequency predicted by Luna's paper is $\omega = 1.2016 - 0.18521i$ with the error bar 0.23% for Real part and 0.11% for Imaginary part.